# Evaluation of Images of Lines from Writer

1. Set the DATA_FOLDER
2. Set the OUTPUT_FILE_NAME
3. Set the CUDA properties
4. Run the jupyter cells.

Output is in a file called OUTPUT_FILE_NAME in the DATA_FOLDER

In [3]:
DATA_FOLDER='../data/sample_writing_examples'
OUTPUT_FILE_NAME='baseline.txt'

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID" 
os.environ["CUDA_VISIBLE_DEVICES"]="1"; 

In [11]:
from __future__ import division
import numpy as np
import os
from keras.utils import to_categorical, np_utils
import pandas as pd

In [6]:
from model import build_model
num_classes = 50
model = build_model(num_classes)

In [7]:
model.load_weights('writer_id_checkpoint.hdf5')

In [14]:
from data_prep import prep_sample

# get slices of normalized image
def generate_predict_data(filename):
    images = []
    targets = []
    prep_sample(filename, 0, images, targets)

    # trim image slices to only see section with road
    image_arrays = np.array(images)

    #reshape image_arrays for feeding in later
    image_arrays = image_arrays.reshape(image_arrays.shape[0], 113, 113, 1)
    #convert to float and normalize
    image_arrays = image_arrays.astype('float32')
    image_arrays /= 255
            
    return image_arrays

In [ ]:
from scipy import stats
    
folders = [DATA_FOLDER]
for folder in folders:
    with open(os.path.join(folder,OUTPUT_FILE_NAME),'w') as fp:
        print(folder)
        for root, dirs, files in os.walk(folder, topdown=False):
            for file in files:
                if file.endswith('png') or file.endswith('jpg'):
                    predictions = model.predict_generator(generate_predict_data(os.path.join(root, file)))
                    #print (predictions)
                    best = stats.mode(np.argmax(predictions,axis=1))[0][0]
                    fp.write(f'{file},{best}\n')